<a href="https://colab.research.google.com/github/rprimi/colB5BERT/blob/main/python/colB5BERT_fine_tuning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **colB5BERT:** Fine tuning colBERT with Big Five dataset

Solução Baseada no notebook do Pedro Genco

In [ ]:
!pip3 install transformers


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/rprimi/colB5BERT.git

%cd /content/colB5BERT
!git pull


Cloning into 'colB5BERT'...
remote: Enumerating objects: 270, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 270 (delta 55), reused 18 (delta 18), pack-reused 196
Receiving objects: 100% (270/270), 32.72 MiB | 42.14 MiB/s, done.
Resolving deltas: 100% (170/170), done.
/content/colB5BERT
Already up to date.


### Libraries

In [ ]:
import sys
sys.path.append('/content/colB5BERT/python/')

import utils
import vsm
import sst

import os
import pandas as pd
import numpy as np
import textwrap
import pickle
import h5py
import logging

from transformers import BertModel, BertTokenizer
from transformers import RobertaModel, RobertaTokenizer

import torch
from torch.nn.functional import cosine_similarity
from tqdm import tqdm

import pandas as pd

from pathlib import Path
from sklearn.model_selection import train_test_split


from transformers import AutoTokenizer
from torch.utils import data
from torch.utils.data import DataLoader
from transformers import BatchEncoding

import torch
from transformers import AutoModel

from transformers import Trainer, TrainingArguments
from transformers import get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup, AdamW
from tqdm.auto import tqdm

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jun 27 12:23:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    46W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/colB5BERT/dataset_test_positive.tsv', sep='\t')
df_train = pd.read_csv('/content/drive/MyDrive/colB5BERT/dataset_train_positive.tsv', sep='\t')

len(df_test)
len(df_train)

# Sample data to test the pipeline

#df_test = df_test.sample(frac=0.08)
#df_train = df_train.sample(frac=0.08)
len(df_test)

df_test = df_test.sample(n = 4000)
df_train = df_train.sample(n = 4000)

In [ ]:
!pip install hnswlib evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.6 MB/s eta 0:00:00
  Created wheel for hnswlib: filename=hnswlib-0.7.0-cp310-cp310-linux_x86_64.whl size=2119608 sha256=33cdccca050cb71341c5aba6da8503663d0f520b9d62c71edf2e1c567702cc60
  Stored in directory: /root/.cache/pip/wheels/8a/ae/ec/235a682e0041fbaeee389843670581ec6c66872db856dfa9a4
Successfully built hnswlib


In [ ]:
import json
import torch
import hnswlib
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from torch import nn
from collections import defaultdict
from evaluate import load
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup

In [ ]:
class Encoder(nn.Module):
  def __init__(self, base_model, use_cls = False):
    super().__init__()
    self.base_model = base_model
    self.use_cls = use_cls

  def forward(self, x):
    embeddings = self.base_model(**x)

    if self.use_cls:
      embeddings = embeddings[0][:, 0, :]
    else:
      embeddings = embeddings[0] #First element of model_output contains all token embeddings
      attention_mask = x["attention_mask"]
      input_mask_expanded = attention_mask.unsqueeze(-1).expand(embeddings.size()).float()
      embeddings = torch.sum(embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    return embeddings

In [ ]:
## Parametros de max_length baseados no notebook do Marcos Piau
class QueriesDataset(Dataset):
  def __init__(self, queries, tokenizer):
    self.queries = queries
    self.tokenizer = tokenizer
    self.cache = {}

  def __len__(self):
    return len(self.queries)

  def __getitem__(self, idx):
    if self.cache.get(idx) is None:
      self.cache[idx] = self.tokenizer(
                            self.queries[idx],
                            padding="max_length",
                            truncation=True,
                            max_length=32,
                            return_tensors='pt'
                        )

    output = {
        "input_ids": self.cache[idx]["input_ids"].squeeze(),
        "attention_mask": self.cache[idx]["attention_mask"].squeeze()
    }
    return output

class DocumentsDataset(Dataset):
  def __init__(self, relevant_documents,  tokenizer):
    self.relevant_documents = relevant_documents
   # self.not_relevant_documents = not_relevant_documents
    self.tokenizer = tokenizer
    self.relevant_cache = {}
  #  self.not_relevant_cache = {}

  def __len__(self):
    return len(self.relevant_documents)

  def __getitem__(self, idx):
    if self.relevant_cache.get(idx) is None:
      self.relevant_cache[idx] = self.tokenizer(
                                    self.relevant_documents[idx],
                                    padding="max_length",
                                    truncation=True,
                                    max_length=512,
                                    return_tensors='pt'
                                )

#    if self.not_relevant_cache.get(idx) is None:
#      self.not_relevant_cache[idx] = self.tokenizer(
#                                      self.not_relevant_documents[idx],
#                                      padding="max_length",
#                                      truncation=True,
#                                      max_length=256,
#                                      return_tensors='pt'
#                                  )

    sample = {
              "relevant_input_ids": self.relevant_cache[idx]["input_ids"].squeeze(),
              "relevant_attention_mask": self.relevant_cache[idx]["attention_mask"].squeeze()
#              "not_relevant_input_ids": self.not_relevant_cache[idx]["input_ids"].squeeze(),
#              "not_relevant_attention_mask": self.not_relevant_cache[idx]["attention_mask"].squeeze(),
             }
    return sample

## Eval and Training loop

In [ ]:
def eval(eval_dataloader_query, eval_dataloader_doc, model_query, model_doc, device):
  loss_val = 0
  model_query.eval()
  model_doc.eval()
  for query_batch, doc_batch in tqdm(zip(eval_dataloader_query, eval_dataloader_doc), total=len(eval_dataloader_query), desc='Validation batches'):
    with torch.no_grad():
      query_input = {
          "input_ids": query_batch["input_ids"].to(device),
          "attention_mask": query_batch["attention_mask"].to(device)
      }
      query_emb = model_query(query_input)

      relevant_input = {
          "input_ids": doc_batch["relevant_input_ids"].to(device),
          "attention_mask": doc_batch["relevant_attention_mask"].to(device)
      }

#      not_relevant_input = {
#          "input_ids": doc_batch["not_relevant_input_ids"].to(device),
#          "attention_mask": doc_batch["not_relevant_attention_mask"].to(device)
#      }

      relevant_embeddings = model_doc(relevant_input)
#      not_relevant_embeddings = model_doc(not_relevant_input)

      ## Split in two parts: dot product of query and relevant, and dot product of query and not rel

      sim = torch.matmul(query_emb, relevant_embeddings.T) # B, H x H, B = B, B
      exp_sim = torch.exp(sim) # B, B
      relevant_sim = torch.diagonal(exp_sim, 0) # B,
      sum_all_sims = torch.sum(exp_sim, dim=1) # B,

      # not_relevant_sim = torch.sum(query_emb * not_relevant_embeddings, dim=1) # B,
      # exp_not_relevant_sim = torch.exp(not_relevant_sim) # B,
      # sum_all_sims = sum_all_sims + exp_not_relevant_sim # B,

      log_loss = -1 * torch.log(relevant_sim / sum_all_sims)
      loss = torch.mean(log_loss)
      loss_val += loss.cpu().item()

  print(f"Eval loss: {loss_val / len(eval_dataloader_query)}")


def train(train_dataloader_query, train_dataloader_doc,
          eval_dataloader_query, eval_dataloader_doc,
          model_query, model_doc, optimizer, scheduler, epochs, device):
  for epoch in tqdm(range(epochs), desc='Epochs'):
    model_query.train()
    model_doc.train()
    train_loss = 0
    for query_batch, doc_batch in tqdm(zip(train_dataloader_query, train_dataloader_doc), total=len(train_dataloader_query), desc=f'Training batches (Epoch {epoch})'):
      optimizer.zero_grad()
      query_input = {
          "input_ids": query_batch["input_ids"].to(device),
          "attention_mask": query_batch["attention_mask"].to(device)
      }
      query_emb = model_query(query_input)

      relevant_input = {
          "input_ids": doc_batch["relevant_input_ids"].to(device),
          "attention_mask": doc_batch["relevant_attention_mask"].to(device)
      }

   #   not_relevant_input = {
   #       "input_ids": doc_batch["not_relevant_input_ids"].to(device),
   #       "attention_mask": doc_batch["not_relevant_attention_mask"].to(device)
   #   }

      relevant_embeddings = model_doc(relevant_input)
   #   not_relevant_embeddings = model_doc(not_relevant_input)

      ## Split in two parts: dot product of query and relevant, and dot product of query and not rel

      sim = torch.matmul(query_emb, relevant_embeddings.T) # B, H x H, B = B, B
      exp_sim = torch.exp(sim) # B, B
      relevant_sim = torch.diagonal(exp_sim, 0) # B,
      sum_all_sims = torch.sum(exp_sim, dim=1) # B,

      # not_relevant_sim = torch.sum(query_emb * not_relevant_embeddings, dim=1) # B,
      # exp_not_relevant_sim = torch.exp(not_relevant_sim) # B,
      # sum_all_sims = sum_all_sims + exp_not_relevant_sim # B,

      log_loss = -1 * torch.log(relevant_sim / sum_all_sims)
      loss = torch.mean(log_loss)
      loss.backward()
      optimizer.step()
      scheduler.step()

      train_loss += loss.cpu().item()

    print(f"Epoch {epoch}")
    print(f"Train loss: {train_loss / len(train_dataloader_query)}")
    eval(eval_dataloader_query, eval_dataloader_doc, model_query, model_doc, device)
  return model_query, model_doc

In [ ]:
params = {
    "use_cls": False,
    "similarity_function": "dot",
    "model_name": 'neuralmind/bert-base-portuguese-cased',
}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(params["model_name"])
model_query = AutoModel.from_pretrained(params["model_name"])
encoder_query = Encoder(model_query, False)

model_doc = AutoModel.from_pretrained(params["model_name"])
encoder_doc = Encoder(model_doc, False)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertModel: ['cls.predicti

In [ ]:
#df = pd.read_csv("https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv", sep="\t", header=None)
#df.columns = ["query", "relevant", "not_relevant"]

#df = df.sample(frac=1.)

#df_train = df.iloc[:10000]
#df_eval = df.iloc[10000:]

#queries_train = df_train['item_pt_text'].tolist()
#docs_train = df_train['texto_dividido'].tolist()

#queries_val = df_test['item_pt_text'].tolist()
#docs_val = df_test['texto_dividido'].tolist()

df_test = pd.read_csv('/content/drive/MyDrive/colB5BERT/dataset_test_positive.tsv', sep='\t')
df_train = pd.read_csv('/content/drive/MyDrive/colB5BERT/dataset_train_positive.tsv', sep='\t')

len(df_train)

df_test = df_test.sample(n = 2400)
df_train = df_train.sample(n = 2400)

2400

In [ ]:
query_dataset_train = QueriesDataset(queries = list(df_train['item_pt_text']), tokenizer = tokenizer)
doc_datasets_train = DocumentsDataset(list(df_train['texto_dividido']),  tokenizer)

query_dataset_eval = QueriesDataset(queries = list(df_test['item_pt_text']), tokenizer = tokenizer)
doc_datasets_eval = DocumentsDataset(list(df_test['texto_dividido']), tokenizer)

In [ ]:
train_query_dataloader = DataLoader(query_dataset_train, batch_size = 32, shuffle=False)
train_doc_dataloader = DataLoader(doc_datasets_train, batch_size = 32, shuffle=False)

eval_query_dataloader = DataLoader(query_dataset_eval, batch_size = 32, shuffle=False)
eval_doc_dataloader = DataLoader(doc_datasets_eval, batch_size = 32, shuffle=False)

In [ ]:
# query_dataset_overfit = QueriesDataset(queries = list(df_train["query"])[:64], tokenizer = tokenizer)
# doc_datasets_overfit = DocumentsDataset(list(df_train["relevant"])[:64], list(df_train["not_relevant"])[:64], tokenizer)

# overfit_query_dataloader = DataLoader(query_dataset_overfit, batch_size = 32, shuffle=False)
# overfit_doc_dataloader = DataLoader(doc_datasets_overfit, batch_size = 32, shuffle=False)

In [ ]:
import gc

torch.cuda.empty_cache()
gc.collect()

39700

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
encoder_doc = encoder_doc.to(device)
encoder_query = encoder_query.to(device)
optim_params = list(encoder_query.parameters()) + list(encoder_doc.parameters())
optimizer = torch.optim.AdamW(optim_params, lr=5e-5)

epochs = 4
num_training_steps = epochs * len(train_query_dataloader)
num_warmup_steps = int(num_training_steps * 0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

encoder_query, encoder_doc = train(train_query_dataloader, train_doc_dataloader,
              eval_query_dataloader, eval_doc_dataloader,
              encoder_query, encoder_doc, optimizer, scheduler, epochs, device)

# encoder_query, encoder_doc = train(overfit_query_dataloader, overfit_doc_dataloader,
#                                    overfit_query_dataloader, overfit_doc_dataloader,
#                                    encoder_query, encoder_doc, optimizer, 100, device)

dir(encoder_query)

type(encoder_query)


encoder_query.save_pretrained('/content/drive/MyDrive/colB5BERT/')
encoder_doc.save_pretrained('/content/drive/MyDrive/colB5BERT/')


__main__.Encoder